# Download User Sessions

Download user sessions from GP3S using the API.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of GP3S Query and is distributed under the terms of the GNU General Public License.

GP3S Query is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GP3S Query is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GP3S Query. If not, see https://www.gnu.org/licenses/.

## API URLs

Azure API hosts the GP3S API calls and returns JSON data

In [1]:
API_URL = 'apimgp3s.azure-api.net'
API_NAME = 'gp3s'

## Import Common Modules

In [2]:
import os
import sys

import http.client
import urllib.parse

import json

## User IDs

TODO - use spot / event rankings to determine list of users to download

In [3]:
userIds = {
    5246: 'Matthew Spooner',
    2640: 'Michael George'
}

## Retrieve User Sessions from the API

In [4]:
def getApiData(url, filename):
    """Get data from the GP3S API"""

    keyPath = os.path.join(projdir, 'keys', 'gp3s')
    with open(keyPath) as f:
        key = f.readline().strip()

    headers = {
        'Ocp-Apim-Subscription-Key': key
    }

    conn = http.client.HTTPSConnection(API_URL)
    conn.request("GET", url, headers=headers)
    response = conn.getresponse()
    rawData = response.read()
    conn.close()

    data = json.loads(rawData)
    data = sorted(data, key=lambda x: (x['session_id'], x['session_id']))
    output = json.dumps(data, indent=2)

    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    with open(filename, "w") as f:
        f.write(output)
        
    return data

In [5]:
def getUserSessions(userId, userName, year, service='UserSessions'):
    """Get user sessions from the GP3S API"""

    print('Downloading {} for {} - {}'.format(service, userName, year))

    params = urllib.parse.urlencode({
        'user_id': userId,
        'year': year,
        'session_id': 0
    })
    
    url = "/{}/{}?{}".format(API_NAME, service, params)
    filename = os.path.join(projdir, 'cache', service, str(userId), str(year) + '.json')

    sessions = getApiData(url, filename)
        
    return sessions

## Download User Sessions

In [6]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

    for userId, userName in userIds.items():
        for year in range(2004, 2023):
            getUserSessions(userId, userName, year)

        print()
        
    print('All done!')



All done!
